# Efficient Yelp API Calls (Core)

For this assignment, you will be working with the Yelp API.

As before, you will use the Yelp API to search your favorite city for a cuisine type of your choice.

Extract all of the results from your search and compile them into one dataframe using a for loop as shown in the lesson "Code for Efficient API Extraction"

Save your notebook, commit the change to your repository and submit the repository URL for this assignment.

### Imports and Functions

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
!pip install tqdm


In [3]:
with open(r'C:\Users\tulan\.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [4]:
# Create an instance with your key
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [7]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    file_exists = os.path.isfile(JSON_FILE)
    ## If it does exist
    if file_exists == True:
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            ## delete file and confirm it no longer exits.
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
            
        else:
            print(f"[i] {JSON_FILE} already exists.")
                
    else:
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)

In [11]:
JSON_FILE = "Data/Results_For_Mexican_SanJose.json"
JSON_FILE

'Data/Results_For_Mexican_SanJose.json'

### Assignment

In [12]:
# Search
LOCATION = 'San Jose, CA'
TERM = 'Mexican'

In [13]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)

[i] Data/Results_For_Mexican_SanJose.json not found. Saving empty list to new file.


In [14]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

- 0 previous results found.


285

In [15]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)



  0%|          | 0/285 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [16]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,dnHSLaWVWnSsOqt5ZaDpsg,añejo-cantina-and-kitchen-san-jose,Añejo Cantina & Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/P3mrJu...,False,https://www.yelp.com/biz/a%C3%B1ejo-cantina-an...,59,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",5.0,"{'latitude': 37.30798313369885, 'longitude': -...","[restaurant_reservation, pickup, delivery]",$$,"{'address1': '3132 Williams Rd', 'address2': N...",+14082613714,(408) 261-3714,11962.616149
1,exmnsc3xUR-GbwppCX_8FA,casa-azteca-milpitas,Casa Azteca,https://s3-media2.fl.yelpcdn.com/bphoto/JD1dET...,False,https://www.yelp.com/biz/casa-azteca-milpitas?...,907,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.0,"{'latitude': 37.431847, 'longitude': -121.908976}","[delivery, pickup]",$$,"{'address1': '20 N Abel St', 'address2': '', '...",+14089460466,(408) 946-0466,17044.165609
2,bDNHsR1Vw4_N5p0bIcNs1A,casa-villa-san-jose,Casa Villa,https://s3-media2.fl.yelpcdn.com/bphoto/Qt5R96...,False,https://www.yelp.com/biz/casa-villa-san-jose?a...,122,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 37.31018244488701, 'longitude': -...","[delivery, pickup]",$$,"{'address1': '840 Willow St', 'address2': 'Ste...",+14083829491,(408) 382-9491,7150.573542
3,Yv0Kxu9DH7WoURxZSwd0lA,el-charrito-salinas-2,El Charrito,https://s3-media3.fl.yelpcdn.com/bphoto/ArKAO2...,False,https://www.yelp.com/biz/el-charrito-salinas-2...,1307,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 36.6790631, 'longitude': -121.659...","[delivery, pickup]",$,"{'address1': '122 W Market St', 'address2': ''...",+18314249446,(831) 424-9446,70155.268869
4,jHzWYtX_79aBuM06V268wQ,adelitas-taqueria-and-bar-san-jose-5,Adelita's Taqueria & Bar,https://s3-media3.fl.yelpcdn.com/bphoto/YEjHjs...,False,https://www.yelp.com/biz/adelitas-taqueria-and...,1759,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.0,"{'latitude': 37.275101, 'longitude': -121.923678}","[delivery, pickup]",$$,"{'address1': '1896 Curtner Ave', 'address2': '...",+14083699905,(408) 369-9905,9808.763069


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,0lFMhtTPRk0BR8Q7H33Q2Q,taqueria-el-buen-sabor-no-2-san-francisco,Taqueria El Buen Sabor no 2,https://s3-media4.fl.yelpcdn.com/bphoto/l4SdA-...,False,https://www.yelp.com/biz/taqueria-el-buen-sabo...,33,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.0,"{'latitude': 37.71199682450079, 'longitude': -...",[delivery],$,"{'address1': '98 Leland Ave', 'address2': '', ...",+14153494134,(415) 349-4134,69469.435321
996,YDcIwDNq6-Zl_G5K517D7g,whipple-taqueria-union-city-2,Whipple Taqueria,https://s3-media2.fl.yelpcdn.com/bphoto/uDxjq4...,False,https://www.yelp.com/biz/whipple-taqueria-unio...,250,"[{'alias': 'mexican', 'title': 'Mexican'}]",3.5,"{'latitude': 37.60567, 'longitude': -122.05131}","[pickup, delivery]",$,"{'address1': '1790 Whipple Rd', 'address2': ''...",+15104891224,(510) 489-1224,40072.426104
997,h1wDLfgEWihWTBJ3z33NLg,damn-fine-san-francisco-2,Damn Fine,https://s3-media2.fl.yelpcdn.com/bphoto/x1A1QU...,False,https://www.yelp.com/biz/damn-fine-san-francis...,194,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 37.76098, 'longitude': -122.4988}",[],$$,"{'address1': '3410 Judah St', 'address2': '', ...",+14159417503,(415) 941-7503,79228.415865
998,-kCMH1ctCUd39mExXkOTbA,hook-fish-at-proof-lab-beer-garden-mill-valley-6,Hook Fish at Proof Lab Beer Garden,https://s3-media2.fl.yelpcdn.com/bphoto/B0q4ew...,False,https://www.yelp.com/biz/hook-fish-at-proof-la...,342,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,"{'latitude': 37.882, 'longitude': -122.52458}",[],$$,"{'address1': '254 Shoreline Hwy', 'address2': ...",+14155694984,(415) 569-4984,90110.874569
999,YJNjFzWA9SS6wuF8BRFdcA,rancho-fresco-mexican-grill-patterson-patterson,Rancho Fresco Mexican Grill - Patterson,https://s3-media1.fl.yelpcdn.com/bphoto/G1EguR...,False,https://www.yelp.com/biz/rancho-fresco-mexican...,39,"[{'alias': 'mexican', 'title': 'Mexican'}]",3.5,"{'latitude': 37.466063, 'longitude': -121.142582}",[pickup],NaN,"{'address1': '1040 W Las Palmas Ave', 'address...",+12098942197,(209) 894-2197,62428.930197
